In [1]:
import pandas as pd
import numpy as np
import warnings
import boto3
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [2]:
client = boto3.client('s3')

In [3]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [4]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/data/feature_prep/')

In [5]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [6]:
paths

['s3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_100D.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_10D.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_10min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_10min_rsi.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_120min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_150D.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_15D.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_15min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_15min_rsi.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_180min.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_1D.parquet'

In [ ]:
def read_and_calculate_rsi(path, window, fields):
    df = pd.read_parquet(path)
    grouped = df.groupby('symbol')
    results = []
    for symbol, group in grouped:
                calculated_fields = []
                for col in eligible_close_fields:
                    for window in [7, 9, 11, 14, 17, 21]:
                        rsi_values = calculate_rsi(group[col], window)
                        group[f'{col}_{window}window_rsi'] = rsi_values
                        calculated_fields.append(f'{col}_{window}window_rsi')
                results.append(group[eligible_close_fields + calculated_fields])
            # Concatenate all the grouped results back into a single DataFrame
            df_concat = pd.concat(results)
            loc = path.replace('.parquet', '_rsi.parquet')
            print(f'Saving to {loc}')
            df_concat.to_parquet(loc)
    
    
for path in paths:
    name = path.split('/')[-1].split('.')[0].split('_')[-1]
    # if 'min' in name and int(name.replace('min', '')) < 100:  # Process files for <100 min
    if 'min' in name:
        if int(name.replace('min', ''))<=10:
            pass
        elif int(name.replace('min', ''))<=30:
            pass
        else:
            pass
    elif 'D' in name:
        if int(name.replace('D', '')) < 100:
            pass
        else: 
            pass

In [8]:
import pandas as pd
from tqdm import tqdm

def calculate_rsi(data, window=14):
    # Calculate price differences
    delta = data.diff()
    # Make two series: one for gains and one for losses
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    # Calculate the Exponential Moving Average of gains and losses
    avg_gain = gain.ewm(alpha=1/window, min_periods=window, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/window, min_periods=window, adjust=False).mean()

    # Calculate the RSI
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

for path in paths:
    name = path.split('/')[-1].split('.')[0].split('_')[-1]
    # if 'min' in name and int(name.replace('min', '')) < 100:  # Process files for <100 min
    if 'min' in name and int(name.replace('min', '')) in [20, 25, 30, 60]:  # Process files for <100 min 10, 15, 1 20 25 2 30 3 5 60
        df = pd.read_parquet(path)
        if 'symbol' in df.columns:
            # Process each symbol separately
            grouped = df.groupby('symbol')
            results = []
            eligible_close_fields = ['close', 'close_ema_2m', 'close_sma_2m']
            for symbol, group in grouped:
                calculated_fields = []
                for col in eligible_close_fields:
                    for window in [7, 9, 11, 14, 17, 21]:
                        rsi_values = calculate_rsi(group[col], window)
                        group[f'{col}_{window}window_rsi'] = rsi_values
                        calculated_fields.append(f'{col}_{window}window_rsi')
                results.append(group[eligible_close_fields + calculated_fields])
            # Concatenate all the grouped results back into a single DataFrame
            df_concat = pd.concat(results)
            loc = path.replace('.parquet', '_rsi.parquet')
            print(f'Saving to {loc}')
            df_concat.to_parquet(loc)
        else:
            print(f"No 'symbol' column in {path}")
    elif '1D' in name:
        # Ignore daily data as specified
        pass


Saving to s3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_20min_rsi.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_25min_rsi.parquet
Saving to s3://sisyphus-general-bucket/AthenaInsights/data/feature_prep/stock_bars_30min_rsi.parquet


KeyError: 'close_ema_2m'

In [ ]:
# def calculate_rsi(data, window=14):
#     # Calculate price differences
#     delta = data.diff()
#     # Make two series: one for gains and one for losses
#     gain = (delta.where(delta > 0, 0)).fillna(0)
#     loss = (-delta.where(delta < 0, 0)).fillna(0)

#     # Calculate the Exponential Moving Average of gains and losses
#     avg_gain = gain.ewm(alpha=1/window, min_periods=window, adjust=False).mean()
#     avg_loss = loss.ewm(alpha=1/window, min_periods=window, adjust=False).mean()

#     # Calculate the RSI
#     rs = avg_gain / avg_loss
#     rsi = 100 - (100 / (1 + rs))
#     return rsi

# for path in paths:
#     name = path.split('/')[-1].split('.')[0].split('_')[-1]
#     if 'min' in name:
#         if int(name.replace('min', '')) < 100:
#             # calculating only for one hour oe lesser time windows
#             print(name)
#             df = pd.read_parquet(path)
#             eligible_close_fields = ['close', 'close_ema_2m' , 'close_sma_2m']
#             calculated_fields = []
#             for col in tqdm(eligible_close_fields):
#                 for window in [7, 9, 11, 14, 17, 21]:
#                     df[f'{col}_{window}window_rsi'] = calculate_rsi(df[col], window)
#                     calculated_fields.append(f'{col}_{window}window_rsi')
#             loc = path.replace('.parquet', '_rsi.parquet')
#             print(f'saving to {loc}')
#             df[eligible_close_fields + calculated_fields].to_parquet(loc)
#     elif '1D' in name:
#         pass

### testing

In [3]:
df1 = pd.read_parquet(f'{s3_prefix}{bucket}/{primary_folder}/data/feature_prep/stock_bars_1min.parquet')
df2 = pd.read_parquet(f'{s3_prefix}{bucket}/{primary_folder}/data/feature_prep/stock_bars_10min.parquet')
# df3 = pd.read_parquet(f'{s3_prefix}{bucket}/{primary_folder}/data/feature_prep/stock_bars_60min.parquet')
# df4 = pd.read_parquet(f'{s3_prefix}{bucket}/{primary_folder}/data/feature_prep/stock_bars_1D.parquet')
# df5 = pd.read_parquet(f'{s3_prefix}{bucket}/{primary_folder}/data/feature_prep/stock_bars_5D.parquet')
# base_df = pd.read_parquet(f'{s3_prefix}{bucket}/{primary_folder}/data/processed/stock_bars_15min.parquet')

In [4]:
x = df1[df1.symbol=='SPY']
y = df2[df2.symbol=='SPY']

### rsi

In [8]:
x.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap,us_eastern_date,...,open_ema_20m,open_sma_20m,high_ema_20m,high_sma_20m,low_ema_20m,low_sma_20m,close_ema_20m,close_sma_20m,volume_ema_20m,volume_sma_20m
us_eastern_timestamp,,,,,,,,,,,,,,,,,,,,,
2019-12-31 19:01:00,SPY,2020-01-01 00:01:00+00:00,322.36,322.36,322.36,322.36,1073.0,23.0,322.36,2019-12-31,...,322.360000,NaN,322.360000,NaN,322.360000,NaN,322.360000,NaN,1073.000000,NaN
2019-12-31 19:11:00,SPY,2020-01-01 00:11:00+00:00,322.38,322.38,322.38,322.38,300.0,6.0,322.38,2019-12-31,...,322.361905,NaN,322.361905,NaN,322.361905,NaN,322.361905,NaN,999.380952,NaN
2019-12-31 19:12:00,SPY,2020-01-01 00:12:00+00:00,322.38,322.38,322.38,322.38,2400.0,16.0,322.38,2019-12-31,...,322.363628,NaN,322.363628,NaN,322.363628,NaN,322.363628,NaN,1132.773243,NaN
2019-12-31 19:14:00,SPY,2020-01-01 00:14:00+00:00,322.35,322.35,322.35,322.35,200.0,1.0,322.35,2019-12-31,...,322.362330,NaN,322.362330,NaN,322.362330,NaN,322.362330,NaN,1043.937696,NaN
2019-12-31 19:18:00,SPY,2020-01-01 00:18:00+00:00,322.38,322.38,322.38,322.38,1910.0,28.0,322.38,2019-12-31,...,322.364013,NaN,322.364013,NaN,322.364013,NaN,322.364013,NaN,1126.419820,NaN


In [24]:
[z for z in x.columns if 'close' in z]

['close',
 'close_ema_2m',
 'close_sma_2m',
 'close_ema_3m',
 'close_sma_3m',
 'close_ema_5m',
 'close_sma_5m',
 'close_ema_10m',
 'close_sma_10m',
 'close_ema_20m',
 'close_sma_20m']